In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
iris_path = "/content/drive/MyDrive/ML/Lab 8/iris.csv"
df = pd.read_csv(iris_path)

In [ ]:
def discretize(df):
  for i in list(df):
    if i != "variety":
      col = df[i]
      print(i)
      bins = pd.cut(col, 3, labels = ["1", "2", "3"]) # Discretizes to bins
      df[i] = bins

In [ ]:
discretize(df)

sepal.length
sepal.width
petal.length
petal.width


In [ ]:
y = df['variety']
X = df.drop(['variety'], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/ML/Lab 8/iris-train.csv")
test = pd.read_csv("/content/drive/MyDrive/ML/Lab 8/iris-train.csv")
y = df['variety']
X = df.drop(['variety'], axis = 1)

In [ ]:
class Node(object):
    def __init__(self, attr):
      self.leaf = False
      self.out = -1
      self.attr_value = attr
      self.children = []
    def set_attribute(self, value):
      self.attr_value = value
    def is_leaf(self, value):
      self.leaf = True
      self.out = value
    def add_child(self, obj):
      self.children.append(obj)


In [ ]:
from math import log2, inf
class DecisionTree:
  def __init__(self):
      self.root_node = Node("")
      self.max_depth = 0
  def entropy(self, X_c, y_c):
    val = 0
    for i in y_c.value_counts():
      val += (i / len(y_c)) * log2(i / len(y_c))
    return val * -1
  def split_by_attr(self, X, y, attr):
    full_dset = X_train.join(y_train)
    vals = []
    for i in range(1, 4):
      x_attr = full_dset.loc[full_dset[attr] == str(i)].drop(['variety'], axis = 1)
      y_attr = full_dset.loc[full_dset[attr] == str(i)]['variety']
      vals.append((x_attr, y_attr))
    return vals
  def recur(self, X_s, y_s, remaining_attributes, node, depth = 0):
    if(depth > self.max_depth or len(remaining_attributes) == 0):
      l = y_s.value_counts()
      outp = l[l == l.max()].index.values[0]
      node.is_leaf(outp)
      return
    best_children = []
    best_attribute = ""
    max_gain = -inf
    parent_info = self.entropy(X_s, y_s)
    if(parent_info == 0):
      l = y_s.unique()
      node.is_leaf(l[0])
      return
    for att in remaining_attributes:
      children = self.split_by_attr(X_s, y_s, att)
      children_entropy = 0
      for i in children:
        X_c, y_c = i[0], i[1]
        child_entropy = self.entropy(X_c, y_c)
        children_entropy += (len(X_c) / len(X_s)) * child_entropy
      print(parent_info, children_entropy, att)
      gain = parent_info - children_entropy
      if gain > max_gain:
        best_attribute = att
        max_gain = gain
        best_children = children
    #print(max_gain, best_attribute)
    remaining_attributes.remove(best_attribute)
    temp = remaining_attributes.copy()
    count = 0
    if max_gain < 0 and depth > 1:
      l = y_s.value_counts()
      outp = l[l == l.max()].index.values[0]
      node.is_leaf(outp)
      return
    for i in best_children:
      #print(depth, count, len(i[0]), temp)
      node.set_attribute(best_attribute)
      n = Node("")
      node.add_child(n)
      self.recur(i[0], i[1], remaining_attributes, n, depth + 1)
      remaining_attributes = temp
      count += 1
  def train(self, X, y):
    self.max_depth = len(list(X))
    self.recur(X, y, list(X), self.root_node)
    return self.root_node
  def predict(self, X):
    preds = []
    for i in X.iterrows():
      preds.append(self.predict_helper(i[1]))
    return pd.Series(preds)
  def predict_helper(self, X):
    return self.predict_recur(X, self.root_node)
  def predict_recur(self, X, node):
    if(node.leaf):
      return node.out
    return self.predict_recur(X, node.children[int(X.loc[node.attr_value]) - 1])
tree = DecisionTree()
r_n = tree.train(X_train, y_train)
y_preds =tree.predict(X_test)

1.5810951599090195 0.9863288986888374 sepal.length
1.5810951599090195 1.338628997553272 sepal.width
1.5810951599090195 0.2517165741980608 petal.length
1.5810951599090195 0.21285276781181822 petal.width
0.4689955935892812 2.9589866960665114 sepal.length
0.4689955935892812 4.015886992659816 sepal.width
0.4689955935892812 0.7551497225941826 petal.length
0.4770713062193297 3.0348581498118072 sepal.length
0.4770713062193297 4.118858454010067 sepal.width
0.2761954276479391 3.8246542787236333 sepal.width
0.1654270339962668 2.886816288845378 sepal.length
0.1654270339962668 3.91793852942421 sepal.width
0.8573309763851856 3.492075645791144 sepal.width
1.2475641589817914 2.8684907090427254 sepal.width


In [ ]:
def printTree(root, level=0):
  if(root.leaf):
    print("  " * level, root.out)
  else:
    print("  " * level, root.attr_value)
  for child in root.children:
      printTree(child, level + 1)
printTree(r_n)

 petal.width
   Setosa
   petal.length
     Setosa
     Versicolor
     Virginica
   sepal.length
     Setosa
     Virginica
     Virginica


In [ ]:
corr, tot = 0, len(y_test)
for i, j in zip(list(y_test), list(y_preds)):
  if i == j:
    corr += 1
print("Testing Accuracy:", corr / tot)
print("\nConfusion Matrix:\n", pd.crosstab(pd.Series(list(y_test)), y_preds))

Testing Accuracy: 1.0

Confusion Matrix:
 col_0       Setosa  Versicolor  Virginica
row_0                                    
Setosa          11           0          0
Versicolor       0          13          0
Virginica        0           0          6


In [ ]:
print(X_test.iloc[0])

sepal.length    2
sepal.width     1
petal.length    3
petal.width     3
Name: 114, dtype: category
Categories (3, object): ['1' < '2' < '3']


In [ ]:
print(y_preds)

Versicolor
